In [6]:
from jupyter_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash as dcc
import dash as html
import plotly.express as px
import dash as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

import animalShelter


# Importing AnimalShelter class 
from animalShelter import AnimalShelter
from aac_crud import AnimalShelter




###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))

#Convert the pymongo cursor into a pandas DataFrame
# df = pd.DataFrame(list(shelter.readAll({})))

#Drop the _id column generated by Mongo
df = df.iloc[:, 1:]



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GS_logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A([
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={
                    'height' : '25%',
                    'width' : '25%',
                    'float' : 'right',
                    'position' : 'relative',
                    'padding-top' : 0,
                    'padding-right' : 0
                })
    ], href='https://www.snhu.edu'),
    html.Br(),
    html.Hr(),
    html.Center(html.B(html.H1('Search and Rescue Candidate Locator'))),
    html.Hr(),
    dcc.Dropdown(
        id='demo-dropdown',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label': 'Water Rescue', 'value': 'WTR'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'MWR'},
            {'label': 'Disaster/Individual Tracking', 'value': 'DIT'}
        ],
        value='NYC'
    ),
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current = 0,
        page_size= 10,
        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side\n",
    html.Div(className='row',
        style={'display' : 'flex'},
        children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
         
            ),
        html.Div(
           id='map-id',
           className='col s12 m6',
            ),
   
        ]),
    html.Br(),
    html.Hr(),
    html.Header("Matthew Bramer")
    ])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('demo-dropdown', 'value')])
def update_dashboard(value):
### FIX ME Add code to filter interactive data table with MongoDB queries
    if (value == 'All'):
        df = pd.DataFrame.from_records(shelter.readAll({}))
    elif (value == 'WTR'):
        df = pd.DataFrame(list(shelter.readAll({"breed" : {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks": {"$lte": 156, "$gte": 26}})))
    elif (value == 'MWR'):
        df = pd.DataFrame(list(shelter.readAll({"breed" : {"$in": ["German Shephard", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks": {"$lte": 156, "$gte": 26}})))
    elif (value == 'DIT'):
        df = pd.DataFrame(list(shelter.readAll({"breed" : {"$in": ["Doberman Pinscher", "German Shephard", "Golden Retriever", "Bloodhound", "Rottweiler"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks": {"$lte": 300, "$gte": 20}})))

        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
        
    return (data,columns)


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
#      ###FIX ME #### add code for chart of your choice (e.g. pie chart) # example
    if (value == 'WTR'):
        names = ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']
        
        labCount = pd.DataFrame(list(shelter.readAll({"breed" : "Labrador Retriever Mix", "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks": {"$lte": 156, "$gte": 26}}))).count()
        chesCount = pd.DataFrame(list(shelter.readAll({"breed" : "Chesapeake Bay Retriever", "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks": {"$lte": 156, "$gte": 26}}))).count()
        NewFndCount = pd.DataFrame(list(shelter.readAll({"breed" : "Newfoundland", "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks": {"$lte": 156, "$gte": 26}}))).count()
        
        numbers = [labCount, chesCount, NewFndCount]
        
    elif (value == 'MWR'):
        names = ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky' , 'Rottweiler']
       
        gShepCount = pd.DataFrame(list(shelter.readAll({"breed" : "German Shephard", "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks": {"$lte": 156, "$gte": 26}}))).count()
        alaskCount = pd.DataFrame(list(shelter.readAll({"breed" : "Alaskan Malamute", "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks": {"$lte": 156, "$gte": 26}}))).count()
        oldSheepCount = pd.DataFrame(list(shelter.readAll({"breed" : "Old English Sheepdog", "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks": {"$lte": 156, "$gte": 26}}))).count()
        siberHCount = pd.DataFrame(list(shelter.readAll({"breed" : "Siberian Husky", "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks": {"$lte": 156, "$gte": 26}}))).count()
        rotCount = pd.DataFrame(list(shelter.readAll({"breed" : "Rottweiler", "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks": {"$lte": 156, "$gte": 26}}))).count()
    
        numbers = [gShepCount, alaskCount, oldSheepCount, rotCount]
        
    elif (value == 'DIT'):
        names = ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']
        
        dobCount = pd.DataFrame(list(shelter.readAll({"breed" : "Doberman Pinscher", "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks": {"$lte": 300, "$gte": 20}}))).count()
        gShepCount = pd.DataFrame(list(shelter.readAll({"breed" : "German Shephard", "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks": {"$lte": 300, "$gte": 20}}))).count()
        bloodCount = pd.DataFrame(list(shelter.readAll({"breed" : "Golden Retriever", "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks": {"$lte": 300, "$gte": 20}}))).count()
        rotCount = pd.DataFrame(list(shelter.readAll({"breed" : "Rottweiler", "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks": {"$lte": 300, "$gte": 20}}))).count()

    return [
       dcc.Graph(
           id='graph_id',
           figure = px.pie(dff, 
                           values='age_upon_outcome_in_weeks', 
                           names='breed',
                           title='Total Age of Particular Breeds'
    
                          ),

       )]   
                                
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.loc[0,'name'])
                ])
            ]) 
        ])
    ]


app

ServerSelectionTimeoutError: localhost:28675: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 648fd4893977731c86557c68, topology_type: Single, servers: [<ServerDescription ('localhost', 28675) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:28675: [Errno 111] Connection refused')>]>